#LangChain
Framework for RAG

-helps augment LLMs
-provides context source (reducing hallucinations)
-works well with unstructured data

# install libraries or dependencies

In [ ]:

# !pip install --upgrade pyarrow
!pip uninstall -y pyarrow==17.0.0
!pip install pyarrow==15.0.0
!pip uninstall -y ragas
!pip install -q ragas

!pip install -q langchain\
langchain_community\
langchain_chroma\
langchain-openai\
langchain-google-vertexai\
langchain-google-genai



# import modules and methods

In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, GoogleGenerativeAI
from langchain_google_vertexai import ChatVertexAI
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

from ragas.metrics import (context_recall, context_precision, faithfulness, answer_correctness,)
from ragas import evaluate

In [ ]:
# accessing hidden api key
from google.colab import userdata
api_key = userdata.get('GOOGLE_API_KEY')

# configure google API key

In [ ]:
# import os
import google.generativeai as genai

# Explicitly configure the API key
genai.configure(api_key = api_key)

# define the data or documents

In [ ]:
data_1 = "https://www.who.int/news-room/fact-sheets/detail/ebola-virus-disease" # fact-sheets
data_2 = "https://www.who.int/health-topics/ebola#tab=tab_1"                    # overview
data_3 = "https://www.who.int/health-topics/ebola#tab=tab_2"                   # symptoms
data_4 = "https://www.who.int/health-topics/ebola#tab=tab_3"                # treatment and prevention
data_5 = "https://www.who.int/news-room/questions-and-answers/item/ebola-virus-disease"     # questions and answers
data_6 = "https://www.who.int/news/item/23-12-2016-final-trial-results-confirm-ebola-vaccine-provides-high-protection-against-disease"  # vaccine testimony
data_7 = "https://www.who.int/news/item/12-04-2019-ebola-vaccine-candidates-reviewed" # vaccine candidate review
data_8 = "https://www.who.int/news/item/26-11-2018-democratic-republic-of-the-congo-begins-first-ever-multi-drug-ebola-trial"   # multi-drug trial
data_9 = "https://www.who.int/news/item/09-12-2022-ebola-trial-candidate-vaccines-arrive-in-uganda-in-record-79-days-after-outbreak-declared"  # vaccines arrive in Uganda
data_10 = "https://www.who.int/emergencies/situations/ebola-uganda-2022"   # uganda 2022 situation or outbreak
data_11 = "https://www.afro.who.int/countries/uganda/news/uganda-declares-end-ebola-disease-outbreak"   #Uganda declare end of outbreak
data_12 = "https://www.who.int/news/item/03-11-2022-global-health-agencies-outline-plan-to-support-ugandan-government-led-response-to-outbreak-of-ebola-virus-disease" # Global health agencies
data_13 = "https://www.afro.who.int/countries/uganda/news/ministry-health-and-development-partners-tour-ebola-treatment-units-strengthen-response-preparedness" # health ministry tours
data_14 = "https://www.afro.who.int/countries/uganda/news/african-health-ministers-take-steps-curb-ebola-disease-outbreak"  # Africa health ministers
data_15 = "https://www.afro.who.int/countries/uganda/news/contact-tracers-and-village-health-teams-take-ebola-uganda?country=879&name=Uganda"  # ugandan contact tracers
data_16 = "https://www.afro.who.int/news/preventive-ebola-vaccination-safeguards-health-workers-democratic-republic-congo"  # vaccine protection for DRC health workers
data_17 = "https://www.afro.who.int/countries/liberia/news/halting-spread-ebola-through-sexual-transmission" # halting sexual spread
data_18 = "https://openwho.org/courses/knowledge-resources-ebola"   # ebola course 1
data_19 = "https://openwho.org/courses/ebola-clinical-management"   # ebola course 2
data_20 = "https://openwho.org/courses/e-protect"      # ebola course 3
data_21 = "https://openwho.org/courses/GO-en"           # ebola course 4
data_22 = "https://www.who.int/emergencies/situations/ebola-%C3%A9quateur-province-democratic-republic-of-the-congo-2022" # DRC 2022 outbreak
data_23 = "https://www.who.int/emergencies/disease-outbreak-news/item/2022-DON411"  # DRC ebola outbreak news
data_24 = "https://www.who.int/emergencies/situations/ebola-2021-nzerekore-guinea" # Guinea outbreak
data_25 = "https://www.cdc.gov/ebola/about/index.html#:~:text=Ebola%20disease%20is%20caused%20by,as%2080%20to%2090%20percent."  # overview CDC
data_26 = "https://www.cdc.gov/ebola/signs-symptoms/index.html"  # symptoms CDC
data_27 = "https://www.cdc.gov/ebola/causes/index.html"  # transmission CDC
data_28 = "https://www.cdc.gov/ebola/hcp/clinical-guidance/clinical-guidance-for-neonates-born-to-patients-with-suspected-or-confirmed-ebola-disease.html" # ebola and neonates CDC
data_29 = "https://www.cdc.gov/ebola/hcp/clinical-guidance/clinical-guidance-for-ebola-in-people-who-are-pregnant.html"  # ebola and pregnancy CDC
data_30 = "https://en.wikipedia.org/wiki/Ebola"      # ebola wikipedia
data_31 = "https://en.wikipedia.org/wiki/Ebola_vaccine"  # ebola vaccine wikipedia
data_32 = "https://en.wikipedia.org/wiki/Research_in_management_of_Ebola"  # ebola research wikipedia



In [ ]:
loader = WebBaseLoader(
    web_paths=(data_1, data_2, data_3, data_4, data_6, data_7, data_8, data_9, data_10, data_11, data_12, data_13, data_14, data_15, data_16, data_17,
               data_22, data_23, data_24, data_25, data_26, data_27, data_28, data_29),   # yet to add wikipedia data and the courses and data_5
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer()
    ),
)
docs = loader.load()
docs

In [ ]:
# refine or format the loaded docs

def clean_docs(docs):
    for doc in docs:
        doc.page_content = doc.page_content.replace("\n", "").replace("\t", "").replace("\r", "")
    return docs


cleaned_docs = clean_docs(docs)


def format_docs(docs):
    return [doc.page_content for doc in docs]


formatted_docs = format_docs(cleaned_docs)
formatted_docs


In [ ]:
def split_docs(doc_to_split):
  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True,
  )
  return text_splitter.split_documents(doc_to_split)


#len(splitted_docs) helps to see length of splitted documents
spliitted_docs = split_docs(cleaned_docs)
spliitted_docs

# create and store embeddings

In [ ]:
def store_embeddings(docs_to_embed):
  vectorstore = Chroma.from_documents(
    documents=docs_to_embed,
    embedding=GoogleGenerativeAIEmbeddings(google_api_key=api_key, model="models/embedding-001"),
  )

  return vectorstore


vectorstore = store_embeddings(spliitted_docs)

# retrieve documents

In [ ]:
retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 3}
        )
user_query = "when did first ebola case happened?"


retrieved_docs = retriever.invoke(user_query)
retrieved_docs

In [ ]:
print(retrieved_docs[0].page_content)

In [ ]:
llm = GoogleGenerativeAI(
    google_api_key=api_key, model="gemini-1.5-flash",
    temperature=0.6
)

In [ ]:
# this prompt helps to formulate query for retrieving documents with past conversation in mind
history_retriever_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",  "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

# use the formulated query for retrieving from past conversation and original documents
history_aware_retriever = create_history_aware_retriever(
    llm,
    retriever,
    history_retriever_prompt
)


In [ ]:
# this prompt tells the llm to answer questions based on a users question and retrieved context with history if there is
main_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """
    You are ChatEbola, an AI assistant specialized in answering questions and offering understanding about Ebola.
    You were developed by Simeon Krah as a RAG system based on google's gemini models.
    Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    {context}
    """),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]

)

# create a chain to specify how the retrieved context is fed into prompt
question_answer_chain = create_stuff_documents_chain(llm, main_prompt)

# create a retrieval chain (putting all together)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [ ]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# conversation chain that takes session id into consideration and interact with history in mind
final_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:

final_rag_chain.invoke(
    {"input": "what is ebola"},
    config={
        "configurable": {"session_id": "any"}
    },  # constructs a key "any" in `store`.
)

In [ ]:
answers = []
context = []

questions = [
    "What is Ebola virus disease?",
    # "How do people become infected with the Ebola virus?",
    "Who is most at risk?",
    "Why are mourners at burial ceremonies considered at risk of contracting Ebola?",
    "Why are health-care workers at greater risk of catching Ebola?",
    # "Can Ebola be transmitted sexually?",
    "What are the typical signs and symptoms of Ebola virus infection?",
    "How long does it take for people to develop symptoms after being infected?"

    ]




for question in questions:
  operation = final_rag_chain.invoke(
    {"input": question},
    config={
        "configurable": {"session_id": "ab13"}
    },  # constructs a key "any" in `store`.
    )
  answers.append(operation["answer"])
  context.append(operation["context"][0].page_content)


print (answers)
print (context)



In [ ]:
import pandas as pd
# context = [["StatisticsAbout WHOAbout WHOPartnershipsCommittees and advisory groupsCollaborating centresTechnical teamsOrganizational structureWho we areOur workActivitiesInitiativesGeneral Programme of WorkWHO AcademyFundingInvestment in WHOWHO FoundationAccountabilityExternal auditFinancial statementsInternal audit and investigations Programme BudgetResults reportsGovernanceGoverning bodiesWorld Health AssemblyExecutive BoardMember States Portal Home/Health topics/Ebola virus disease World Bank Group/ V.Tremeau Community representatives come to visit a family in the outskirts of Beni to raise awareness about Ebola. ©Credits Ebola virus disease Overview Ebola virus disease (EVD), formerly known as Ebola haemorrhagic fever, is a severe, often fatal illness affecting humans and other primates.The virus is transmitted to people from wild animals (such as fruit bats, porcupines and non-human primates) and then spreads in the"], ["bbaba"], ["ebola"], ["ebola virus"]]
data = {'question': questions, 'retrieved_contexts': context, 'answer': answers}
df = pd.DataFrame(data)
df["ground_truth"] = [
     """Ebola virus disease (formerly known as Ebola haemorrhagic fever) is a rare but severe, often fatal , often fatal illness,
        with a death rate of up to 90% in humans caused by the Ebola virus, a member of the filovirus family. Death rates have varied from 25% to 90% in past outbreaks.""",

  # """People become infected with Ebola either through contact with infected animals (usually following butchering, cooking or eating) or through contact with the bodily
  # fluids of infected humans. Most cases are caused by human to human transmission which occurs when blood or other bodily fluids or secretions (stool, urine, saliva,
  #  semen) of infected people enters a healthy person’s body through broken skin or mucous membranes.
  # Infection can also occur if the broken skin or the mucous membranes of a healthy person comes into contact with items or environments contaminated with bodily fluids
  # from an infected person. These may include soiled clothing, bed linen, gloves, protective equipment and medical waste such as used hypodermic syringes.""",

   """During an outbreak, those at higher risk of infection are:  health workers;   family members or others in close contact with infected people;
    mourners who have direct contact with bodies during burial rituals.""",

  """Levels of Ebola virus remain high after death, thus bodies of those who have died from Ebola virus disease must be handled only by people wearing appropriate
  personal protective equipment and must be buried immediately. WHO advises that bodies of people who may have died from Ebola virus disease should be handled
  only by trained burial teams, who are equipped to properly bury the dead, safely and with dignity.""",

   """Health-care workers are at greater risk of infection if they are not wearing correct personal protective equipment (PPE) or are not applying infection
    prevention and control (IPC) measures when caring for patients. All health-care providers working at all levels of the health system – hospitals, clinics
    and health posts – should be fully informed about the disease and its mode of transmission and should follow recommended precautions strictly.""",

    # """Sexual transmission of the Ebola virus, from males to females, is a strong possibility, but has not yet been proven. Less probable, but theoretically possible, is female to male transmission.
    #  More surveillance data and research are needed on the risks of sexual transmission, and particularly on the prevalence of viable and transmissible virus in
    #  semen over time.""",

     """Ebola symptoms vary but sudden onset of fever, intense weakness, muscle pain, headache and sore throat are commonly experienced at the beginning of the disease
      (‘the dry phase’). As the disease progresses, people commonly develop vomiting and diarrhoea (‘the wet phase’), rash, impaired kidney and liver function,
      and in some cases, both internal and external bleeding.""",

     """The incubation period, or the time interval from infection to onset of symptoms, is from 2 to 21 days. People are not contagious until they develop symptoms.
     Ebola virus disease infections can only be confirmed through laboratory testing."""
]

df["retrieved_contexts"] = df["retrieved_contexts"].apply(lambda x : [x])
# df["ground_truth"] = df["ground_truth"].apply(lambda x : [x])
df



In [ ]:
# !pip install -qU datasets
from datasets import Dataset
eval_dataset = Dataset.from_pandas(df)
eval_dataset

In [ ]:
scores = evaluate(
    eval_dataset,
    metrics=[context_recall, context_precision, faithfulness, answer_correctness],
    llm = llm,
    embeddings=GoogleGenerativeAIEmbeddings(google_api_key=api_key, model="models/embedding-001"),
)


scores

In [ ]:
# # this prompt tells the llm to answer questions based on a users question and retrieved context with history if there is
# main_prompt = ChatPromptTemplate.from_template(
#     """
#     You are ChatEbola, an AI assistant specialized in answering questions and offering understanding about Ebola.
#     Usually, use the following pieces of retrieved context to answer the question.
#     {context}

#     When asked something else or something not in the retrieved document, say you do not know.
#     If you are asked something very sensitive that you think direct advice is needed from a medical doctor,
#     politely tell the user to visit the hospital or consult a doctor.
#     Though you're answering based on the context given, you do not necessary have to always say based on the context or material or document while giving the answer.
#     You were developed by Simeon Krah as a RAG system based on google's gemini models.

# """
# )

In [ ]:
# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
# chain = (
#     {"question": RunnablePassthrough(), "context": retriever | format_docs}
#     | prompt
#     | llm
#     | StrOutputParser()
# )

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# # scores data
# data = {
#   "Context recall": 91,
#   "Context precision": 93,
#   "Faithfulness": 97,
#   "Answer correctness": 80,
# }

# # Convert the dictionary to two lists: metric names and their corresponding values
# metrics = list(data.keys())
# values = list(data.values())

# # Create a bar plot using Seaborn
# plt.figure(figsize=(8, 6))
# sns.barplot(x=metrics, y=values, hue= metrics)

# # Add title and labels
# plt.title('Model Evaluation Metrics', fontsize=16)
# plt.xlabel('Metrics', fontsize=12)
# plt.ylabel('Scores (%)', fontsize=12)

# # Display the plot
# plt.show()
